In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm_notebook as tqdm
import time
import warnings;warnings.simplefilter('ignore')
os.listdir('../data')

['dataset.pkl',
 'PA_201605-2020001更新R8(20200423).csv',
 'tag盤點表20200423.xlsx',
 'PA_201605-2020001更新R9(20200508).csv',
 'train_cleaned.csv',
 'tag_cleaned.csv']

# load raw df

In [2]:
df = pd.read_csv('../data/PA_201605-2020001更新R9(20200508).csv',
                 index_col=0,
                 parse_dates=True,
                 encoding='latin-1')
print(df.shape)
df.head()

(193824, 97)


,MLPAP_TI-0706B02,MLPAP_TI-0706B03,MLPAP_TI-0706B04,MLPAP_TI-0706B05,MLPAP_TI-0706B06,MLPAP_TI-0706B07,MLPAP_TI-0706B08,MLPAP_TI-0706B09,MLPAP_TI-0706B10,MLPAP_TI-0706B11,...,MLPAP_FR-0791.PV,MLPAP_FI-V211.PV,MLPAP_FI-V211.PV(Âà´««á),MLPAP_PI-0658.PV,MLPAP_TIC-0757.PV,MLPAP_V225-PHD,MLPAP_PI-0771.PV,MLPAP_TRC-0747.PV,Unnamed: 96,MLPAP_TRC-0497.PV
2016-05-09 00:00:00,400,401.957,400.752,402.251,401.149,401,400.05,403,402,401,...,6.48477,[-11059] No Good Data For Calculation,#VALUE!,-1,1.63519,0.009,-1.38272,133.1,NaN,8224
2016-05-09 00:10:00,400,401.943,400.95,402.249,401,401,400.25,403,402,401,...,6.44989,[-11059] No Good Data For Calculation,#VALUE!,-1,1.62501,0.009,-1.38273,133.1,NaN,8226
2016-05-09 00:20:00,400,401.9,400.9,402.101,401,401,400.1,403,402,401,...,6.4405,[-11059] No Good Data For Calculation,#VALUE!,-0.898558,1.64978,0.009,-1.4,133.1,NaN,8213.34
2016-05-09 00:30:00,400,401.95,400.9,402.3,401,401,400.145,403,402,401,...,6.46365,[-11059] No Good Data For Calculation,#VALUE!,-0.50901,1.63519,0.009,-1.4,133.2,NaN,8250.89
2016-05-09 00:40:00,400,402,400.85,402.176,401,401,400.405,403,402,401,...,6.4148,[-11059] No Good Data For Calculation,#VALUE!,-0.992771,1.65426,0.009,-1.3,133.1,NaN,8218.33


# load raw tag

In [3]:
tag = pd.read_excel('../data/tag盤點表20200423.xlsx')
print(tag.shape)
tag.head()

(94, 10)


,item,TAG,chinese,character,unit,category,period,min,max,推測顯著性(依製程經驗)
0,1.0,MLPAP_TJC-0627.PV,OX預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,140,160,***
1,2.0,MLPAP_TRC-0610.PV,空氣預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,160,200,***
2,3.0,MLPAP_FR-0632.PV,汽機風量,流量,KNM3/HR,可控制因子,1分鐘2筆,30,90,****
3,4.0,MLPAP_FQ-0619.PV,OX入料量,流量,kg/HR,可控制因子,1分鐘2筆,2000,8000,****
4,5.0,MLPAP_TIC-0757A.PV,反應槽鹽浴溫度,溫度,℃,可控制因子,1分鐘2筆,345,365,****


In [4]:
stream_list = tag[tag.chinese.str.contains('蒸氣')==True].TAG.tolist()
stream_list

['MLPAP_FR-0755PV(轉換後)', 'MLPAP_FR-0791.PV', 'MLPAP_FI-V211.PV(轉換後)']

# clean df and tag 

In [5]:
# rename
df.columns = [str(i).replace('TI','TJ') for i in df.columns]
tag.TAG = [str(i).replace('TI','TJ') for i in tag.TAG]
df.columns = [i.replace('Âà´««á','轉換後')for i in df.columns]

# drop not need columns
df = df.drop(['MLPAP_FR-0755.PV','MLPAP_FI-V211.PV'],axis=1)

# extend tag
temp = tag[tag['TAG'] == tag['TAG'][tag['TAG'].str.contains('0757A')].values[0]].copy()
temp['TAG'] = 'MLPAP_TJ-0757B.PV'
tag = tag.append(temp).reset_index(drop=True)

In [6]:
print('only in tag:',set(tag['TAG'])-set(df.columns))
print('only in df:',set(df.columns)-set(tag['TAG']))

only in tag: {'nan', '第二模型使用'}
only in df: {'Unnamed: 96', 'MLPAP_TRC-0497.PV'}


# convert time

In [7]:
def convert_time(x):
    a,b = str(x).split('*')
    return float(a)*float(b)
df['DATA OF USE'] = [convert_time(i) for i in df['DATA OF USE']]
df[['DATA OF USE']].sample(5)

,DATA OF USE
2020-01-04 19:20:00,0.914940
2016-12-18 10:30:00,0.153044
2017-11-13 14:00:00,0.379171
2016-07-01 14:30:00,0.036720
2016-06-30 01:20:00,0.035659


# replace error value

In [8]:
df = df.replace('[-11059] No Good Data For Calculation',np.nan)
df = df.replace('Scan Off',np.nan)
df = df.replace('Not Connect',np.nan)
df = df.replace('I/O Timeout',np.nan)
df = df.replace('Bad',np.nan)
df = df.replace('#VALUE!',np.nan)
df = df.astype('float32')

# spec fliter

In [9]:
for i in tqdm(set(df.columns)&set(tag.TAG)):
    try:
        lb = float(tag[tag['TAG']==i]['min'].values[0])
        ub = float(tag[tag['TAG']==i]['max'].values[0])
        out = (df[i]<lb)|(df[i]>ub)
        
        # 重點
        df.loc[out,i] = np.nan
    
    except:
        print(i)
df.head()

DATA OF USE



,MLPAP_TJ-0706B02,MLPAP_TJ-0706B03,MLPAP_TJ-0706B04,MLPAP_TJ-0706B05,MLPAP_TJ-0706B06,MLPAP_TJ-0706B07,MLPAP_TJ-0706B08,MLPAP_TJ-0706B09,MLPAP_TJ-0706B10,MLPAP_TJ-0706B11,...,MLPAP_FR-0755PV(轉換後),MLPAP_FR-0791.PV,MLPAP_FI-V211.PV(轉換後),MLPAP_PI-0658.PV,MLPAP_TJC-0757.PV,MLPAP_V225-PHD,MLPAP_PI-0771.PV,MLPAP_TRC-0747.PV,Unnamed: 96,MLPAP_TRC-0497.PV
2016-05-09 00:00:00,400.0,401.957123,400.751862,402.250793,401.148773,401.0,400.050079,403.0,402.0,401.0,...,4.045683,NaN,NaN,NaN,1.635186,0.009,NaN,133.100006,NaN,8224.000000
2016-05-09 00:10:00,400.0,401.942719,400.949890,402.249146,401.000000,401.0,400.249786,403.0,402.0,401.0,...,4.152250,NaN,NaN,NaN,1.625011,0.009,NaN,133.100006,NaN,8226.000000
2016-05-09 00:20:00,400.0,401.900146,400.899780,402.101257,401.000000,401.0,400.100159,403.0,402.0,401.0,...,4.202449,NaN,NaN,NaN,1.649780,0.009,NaN,133.100006,NaN,8213.338867
2016-05-09 00:30:00,400.0,401.949921,400.900208,402.300323,401.000000,401.0,400.144684,403.0,402.0,401.0,...,4.146723,NaN,NaN,NaN,1.635187,0.009,NaN,133.199997,NaN,8250.892578
2016-05-09 00:40:00,400.0,402.000000,400.850037,402.176056,401.000000,401.0,400.404999,403.0,402.0,401.0,...,4.011298,NaN,NaN,NaN,1.654259,0.009,NaN,133.100006,NaN,8218.332031


# become stable

In [10]:
df = df.rolling(6).median()#主要是平滑

In [11]:
df[stream_list].isnull().sum().sort_values(ascending=False)

MLPAP_FI-V211.PV(轉換後)    117707
MLPAP_FR-0791.PV          51816
MLPAP_FR-0755PV(轉換後)       1567
dtype: int64

# save

In [12]:
df.to_csv('../data/train_cleaned.csv')
tag.to_csv('../data/tag_cleaned.csv')
print(time.ctime())

Fri May 22 10:53:27 2020
